In [ ]:
from snp_query_box.dsnp_transform import ictCaseNoteTransform
#for dev purpose
import getpass
import os
USERNAME = os.getenv('USERNAME').lower()
#PASSWORD = getpass.getpass()
new_pull = True

In [ ]:
# import python packages
import sys
if sys.version_info[1]==12:
    os.add_dll_directory(r'C:/Program Files/Python/312/Lib/site-packages/clidriver/bin')
else:
    pass
import pandas as pd
import datetime as dt
import time
import warnings
from dateutil.relativedelta import relativedelta
from snp_query_box import DsnpHelperFunction, snpQueries
from snp_query_box.dsnp_transform import dsnpTransform

warnings.filterwarnings("ignore")
pd.set_option("display.max_columns", None)
print(f"User: {USERNAME} Started creating pull_member_ict_case_note_bq in the shared storage")

In [ ]:
import pkg_resources
pkg_resources.get_distribution("snp_query_box").version

In [ ]:
# set the starting time
start_time = time.time()

In [ ]:
# calculate the dates
today = dt.datetime.today()
today_str = today.strftime('%Y-%m-%d')
last_date_of_last_month_str = DsnpHelperFunction.last_date_of_last_month(today_str, output_type="string")
last_date_of_last_month = DsnpHelperFunction.last_date_of_last_month(today_str)
first_date_of_month = dt.datetime.today().replace(day=1)
first_date_of_last_month = first_date_of_month - relativedelta(months=1)
call_log_date2 = pd.to_datetime(first_date_of_last_month).date()
print(call_log_date2)
call_log_date1 = pd.to_datetime(last_date_of_last_month + dt.timedelta(days=1) - relativedelta(years = 1)).date()
print(call_log_date1)

reporting_end_date = last_date_of_last_month_str
print(reporting_end_date)

In [ ]:
# set the storage and output path
storage_path = r'//mbip/medicarepBI/Projects/COE/DSNP/dsnp_data_storage/monthly_flat_files'
output_path = f'{storage_path}/{reporting_end_date}'

isExist = os.path.exists(output_path)
print(output_path)
if not isExist:
    # Create a new directory because it does not exist
     os.makedirs(output_path)
     print("The new directory is created!")
else:
      print("The folder already exist")
print("Output files will sit in the folder above.")

In [ ]:
#locate data dump folder
if new_pull:
    pull_member_ict_case_note_bq = snpQueries.pull_member_ict_case_note_bq()
    pull_member_ict_case_note_bq, pull_member_ict_supplemental_assessment_bq = ictCaseNoteTransform.splitCaseNoteSupplementalAssessment(pull_member_ict_case_note_bq)
    pull_member_ict_case_note_bq.to_parquet(f"{output_path}/pull_member_ict_case_note_bq.parquet")
    pull_member_ict_supplemental_assessment_bq.to_parquet(f"{output_path}/pull_member_ict_supplemental_assessment_bq.parquet")
else:
    try:
        pull_member_ict_case_note_bq = pd.read_parquet(f"{output_path}/pull_member_ict_case_note_bq.parquet")
        pull_member_ict_supplemental_assessment_bq = pd.read_parquet(f"{output_path}/pull_member_ict_supplemental_assessment_bq.parquet")

    except:
        pull_member_ict_case_note_bq = snpQueries.pull_member_ict_case_note_bq()
        pull_member_ict_case_note_bq, pull_member_ict_supplemental_assessment_bq = ictCaseNoteTransform.splitCaseNoteSupplementalAssessment(pull_member_ict_case_note_bq)
        pull_member_ict_case_note_bq.to_parquet(f"{output_path}/pull_member_ict_case_note_bq.parquet")
        pull_member_ict_supplemental_assessment_bq.to_parquet(f"{output_path}/pull_member_ict_supplemental_assessment_bq.parquet")


In [ ]:
print(pull_member_ict_case_note_bq.shape)
print(pull_member_ict_case_note_bq.head(3))

In [ ]:
pull_member_ict_case_note_bq.groupby("mbr_src_member_id").size().sort_values()

In [ ]:
# output the process time
print("Process time -- %s seconds" % (time.time() - start_time))